In [1]:
shop_data = spark.read.parquet("gs://dsp-bigdata/tables_prod/parquet_log_results/pt=latest").filter("advertiser_app_id = 184")


In [8]:
shop_data

DataFrame[bid_id: string, req_date: string, req_hour: string, bidder_id: string, req_time: timestamp, bid_time: timestamp, imp_flag: int, imp_time: timestamp, click_flag: int, click_time: timestamp, conv_flag: int, conv_time: timestamp, adexchange: string, req_id: string, req_at: int, is_app: boolean, is_banner: boolean, is_video: boolean, app_name: string, publisher_app_id: string, publisher_id: string, publisher_name: string, publisher_app_bundle: string, imp_bid_floor: double, device_lang: string, device_ip: string, device_os: string, device_osv: string, device_hwv: string, device_ifa: string, device_ifv: string, carrier: string, connection_type: int, device_type: int, is_tablet: boolean, release_date: string, form_factor: string, screen_height: int, screen_width: int, nfc_support: boolean, release_msrp: string, device_model: string, device_make: string, user_id: string, country: string, region: string, city: string, isp: string, latitude: float, longitude: float, timezone: string, 

In [3]:
data = shop_data.select("click_flag",
                        "publisher_app_bundle",
                        "is_banner",
                        "device_lang",
                        "device_os",
                        "connection_type",
                        "is_tablet",
                        "release_date",
                        "screen_height",
                        "nfc_support",
                        "city",
                        "campaign_id",
                        'adv_impression_count_hourly',
                        'adv_impression_count_daily',
                        'adv_impression_count_weekly',
                        'adv_impression_count_monthly',
                        'adv_impression_count_lifetime',
                        'adv_click_count_hourly',
                        'adv_click_count_daily',
                        'adv_click_count_weekly',
                        'adv_click_count_monthly',
                        'adv_click_count_lifetime',
                        'all_impression_count_hourly',
                        'all_impression_count_daily',
                        'all_impression_count_weekly',
                        'all_impression_count_monthly',
                        'all_impression_count_lifetime')

In [4]:
data = data.withColumnRenamed("click_flag","label")

In [5]:
cates_cols = ["publisher_app_bundle","is_banner","device_lang","device_os","is_tablet","release_date","nfc_support","city"]

In [6]:
from pyspark.sql.functions import col, split, size
from pyspark.sql.types import StringType
for i in cates_cols:
    data = data.withColumn(i, col(i).cast(StringType()))

In [7]:
data.cache()
data.count()

23/10/08 09:40:14 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


148948182

In [13]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from xgboost.spark import SparkXGBClassifier
from pyspark.ml.linalg import Vectors

In [14]:
string_indexer = StringIndexer(inputCol="is_banner", outputCol="is_bannerIndex")
indexers = [StringIndexer(inputCol=feature, outputCol=feature + '_index', handleInvalid='keep')
            for feature in cates_cols]
for i in indexers:
    m = i.fit(data)
    datat = m.transform(data)

# 拟合 StringIndexer 模型并进行转换
# model = string_indexer.fit(data)
# indexed_df = model.transform(data)

# 显示转换后的 DataFrame
datat.select(feature + '_index' for feature in cates_cols).show()

TypeError: Invalid argument, not a string or column: <generator object <genexpr> at 0x7f3aad3cb270> of type <class 'generator'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [ ]:
[feature + '_index' for feature in cates_cols]

In [ ]:
datat.select("publisher_app_bundle_index","is_banner_index","device_lang_index","device_os_index","is_tablet_index","release_date_index","nfc_support_index","city_index").show()

In [50]:
indexed_df.groupby("release_dateIndex").count().show(1000)

+-----------------+-------+
|release_dateIndex|  count|
+-----------------+-------+
|              8.0| 483483|
|             67.0|  24428|
|             70.0|  22626|
|              0.0|3208705|
|             69.0|  23045|
|              7.0| 505487|
|            112.0|    302|
|             88.0|   2614|
|            108.0|    396|
|            128.0|     15|
|            133.0|      3|
|            124.0|     28|
|             49.0|  57571|
|             98.0|   1097|
|             29.0| 179196|
|             64.0|  28356|
|             75.0|  12074|
|            101.0|    726|
|            107.0|    436|
|            116.0|    164|
|             47.0|  86168|
|             42.0| 131418|
|             44.0| 104986|
|             62.0|  35350|
|             35.0| 157306|
|             96.0|   1376|
|             18.0| 283743|
|             80.0|   6282|
|             86.0|   2996|
|              1.0|1012302|
|             39.0| 135961|
|             94.0|   1815|
|            104.0| 

In [51]:
# assembler = VectorAssembler(inputCols=["is_bannerIndex","all_impression_count_weekly","release_dateIndex"],
#                             outputCol='features')
# vec_data = assembler.transform(indexed_df)

# vec_data.show()

IllegalArgumentException: is_bannerIndex does not exist. Available: label, is_banner, device_lang, device_os, connection_type, is_tablet, release_date, screen_height, nfc_support, city, campaign_id, adv_impression_count_hourly, adv_impression_count_daily, adv_impression_count_weekly, adv_impression_count_monthly, adv_impression_count_lifetime, adv_click_count_hourly, adv_click_count_daily, adv_click_count_weekly, adv_click_count_monthly, adv_click_count_lifetime, all_impression_count_hourly, all_impression_count_daily, all_impression_count_weekly, all_impression_count_monthly, all_impression_count_lifetime, release_dateIndex

In [16]:
assembler = VectorAssembler(inputCols=["all_impression_count_weekly"],
                            outputCol='features')
vec_data = assembler.transform(data)

vec_data.show()

+-----+--------------------+---------+-----------+---------+---------------+---------+--------------+-------------+-----------+------------------+-----------+---------------------------+--------------------------+---------------------------+----------------------------+-----------------------------+----------------------+---------------------+----------------------+-----------------------+------------------------+---------------------------+--------------------------+---------------------------+----------------------------+-----------------------------+--------+
|label|publisher_app_bundle|is_banner|device_lang|device_os|connection_type|is_tablet|  release_date|screen_height|nfc_support|              city|campaign_id|adv_impression_count_hourly|adv_impression_count_daily|adv_impression_count_weekly|adv_impression_count_monthly|adv_impression_count_lifetime|adv_click_count_hourly|adv_click_count_daily|adv_click_count_weekly|adv_click_count_monthly|adv_click_count_lifetime|all_impression

In [17]:
vec_data.select("features").show()

+--------+
|features|
+--------+
|   [0.0]|
|   [2.0]|
|   [1.0]|
|   [1.0]|
|   [1.0]|
|   [0.0]|
|   [2.0]|
|   [4.0]|
|   [2.0]|
|   [2.0]|
|   [2.0]|
|  [-1.0]|
|  [32.0]|
|   [1.0]|
|   [4.0]|
|   [4.0]|
|   [8.0]|
|   [8.0]|
|   [8.0]|
|   [8.0]|
+--------+
only showing top 20 rows



In [18]:
xgb_classifier = SparkXGBClassifier(max_depth=5, missing=0.0,num_workers=6,n_estimators=1,features_col="features",eval_metric='logloss',)

In [19]:
trainData, testData = vec_data.randomSplit([0.7, 0.3], seed=123)


In [20]:
model= xgb_classifier.fit(trainData)

2023-10-08 03:31:15,878 INFO XGBoost-PySpark: _fit Running xgboost-2.0.0 on 6 workers with
	booster params: {'objective': 'binary:logistic', 'device': 'cpu', 'max_depth': 5, 'eval_metric': 'logloss', 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 1}
	dmatrix_kwargs: {'nthread': 1, 'missing': 0.0}
2023-10-08 03:32:18,959 INFO XGBoost-PySpark: _fit Finished xgboost training!   


In [21]:
predictions = model.transform(testData)


In [22]:
predictions.select("all_impression_count_weekly","features","rawPrediction","prediction","probability").show(truncate=False)

+---------------------------+--------+----------------------------------------+----------+----------------------------------------+
|all_impression_count_weekly|features|rawPrediction                           |prediction|probability                             |
+---------------------------+--------+----------------------------------------+----------+----------------------------------------+
|-1                         |[-1.0]  |[0.7046330571174622,-0.7046330571174622]|0.0       |[0.6692141890525818,0.3307858109474182] |
|2                          |[2.0]   |[0.8384885191917419,-0.8384885191917419]|0.0       |[0.6981468200683594,0.301853209733963]  |
|1                          |[1.0]   |[0.8042923212051392,-0.8042923212051392]|0.0       |[0.6908918619155884,0.30910810828208923]|
|0                          |[0.0]   |[0.7301314473152161,-0.7301314473152161]|0.0       |[0.674834132194519,0.32516589760780334] |
|0                          |[0.0]   |[0.7301314473152161,-0.730131447315216

In [23]:
model.get_feature_importances()

{'f0': 9.0}

In [9]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from xgboost.spark import SparkXGBClassifier
from pyspark.ml.linalg import Vectors
# from pyspark.ml.classification import GBTClassifier

# Assuming you have a DataFrame called 'data' with your input features and target variable
# 'categorical_features' is a list of categorical feature column names
# 'label' is the target variable column name

# Step 1: StringIndexer for categorical features
indexers = [StringIndexer(inputCol=feature, outputCol=feature + '_index', handleInvalid='keep')
            for feature in cates_cols]

# Step 2: OneHotEncoder for indexed categorical features
# encoders = [OneHotEncoder(inputCol=feature + '_index', outputCol=feature + '_encoded')
#             for feature in cates_cols]

# Step 3: Assemble feature vectors
assembler = VectorAssembler(inputCols=[feature + '_index' for feature in cates_cols],
                            outputCol='features')

# Step 4: Create the GBTClassifier
# gbt = GBTClassifier(labelCol="label", featuresCol='features', maxIter=10)
xgb_classifier = SparkXGBClassifier(max_depth=5, missing=0.0,num_workers=6,
    eval_metric='logloss')


# Step 5: Build the pipeline
pipeline = Pipeline(stages=indexers + [assembler, xgb_classifier])

# Step 6: Split the data into training and test sets
trainData, testData = data.randomSplit([0.7, 0.3], seed=123)

# Step 7: Fit the pipeline model
model = pipeline.fit(trainData)

# # Step 8: Extract feature importance
# feature_importance = model.stages[-1].featureImportances

# # Step 9: Display feature importance scores
# for idx, feature in enumerate(use_fea_cols_nl):
#     print("Categorical Feature '{}': Importance = {}".format(feature, feature_importance[idx]))


23/10/08 09:47:42 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.5 MiB
23/10/08 09:48:22 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.5 MiB
/opt/conda/miniconda3/lib/python3.8/site-packages/xgboost/sklearn.py:782: UserWarning: Loading a native XGBoost model with Scikit-Learn interface.
  warnings.warn("Loading a native XGBoost model with Scikit-Learn interface.")


In [29]:
model.stages[7]

StringIndexerModel: uid=StringIndexer_331a9cc218bf, handleInvalid=keep

In [12]:
from pyspark2pmml import PMMLBuilder

pmmlBuilder = PMMLBuilder(spark.sparkContext, trainData, model)

pmmlBuilder.buildFile("./xgb.pmml")

AttributeError: 'SparkXGBClassifierModel' object has no attribute '_to_java'

In [20]:
model.stages[-1]

{'f0': 1286.0,
 'f1': 207.0,
 'f2': 734.0,
 'f4': 18.0,
 'f5': 500.0,
 'f6': 49.0,
 'f7': 296.0}

In [21]:
predictions = model.transform(testData)


In [14]:
cates_cols

['publisher_app_bundle',
 'is_banner',
 'device_lang',
 'device_os',
 'is_tablet',
 'release_date',
 'nfc_support',
 'city']

In [15]:
predictions.select("features").show(truncate=False)


23/09/28 07:48:47 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.7 MiB


+-------------------------------------+
|features                             |
+-------------------------------------+
|(8,[0,2,7],[2777.0,1.0,399.0])       |
|(8,[0,2,7],[40.0,1.0,967.0])         |
|(8,[0,2,7],[40.0,1.0,1.0])           |
|(8,[0,2,7],[40.0,1.0,36.0])          |
|[40.0,0.0,1.0,0.0,0.0,63.0,1.0,12.0] |
|(8,[0,2,5,7],[40.0,1.0,12.0,360.0])  |
|(8,[0,2,5,7],[40.0,1.0,1.0,53.0])    |
|(8,[0,2,5,7],[40.0,1.0,16.0,883.0])  |
|(8,[0,2,5,7],[40.0,1.0,33.0,1845.0]) |
|[40.0,0.0,1.0,0.0,0.0,20.0,1.0,355.0]|
|[40.0,0.0,1.0,0.0,0.0,11.0,1.0,13.0] |
|(8,[0,2,5,7],[40.0,1.0,51.0,119.0])  |
|(8,[0,2,5,7],[40.0,1.0,6.0,103.0])   |
|[40.0,0.0,1.0,0.0,0.0,22.0,1.0,33.0] |
|(8,[0,2,5,7],[40.0,1.0,16.0,248.0])  |
|(8,[0,2,5,7],[40.0,1.0,33.0,12.0])   |
|(8,[0,2,7],[40.0,4.0,29.0])          |
|(8,[0,2,5,7],[40.0,4.0,7.0,744.0])   |
|(8,[0,2,5,7],[40.0,4.0,21.0,35.0])   |
|[40.0,0.0,4.0,0.0,0.0,11.0,1.0,26.0] |
+-------------------------------------+
only showing top 20 rows



In [24]:
model.stages[-1].write().overwrite().save("gs://dsp_alg_test/tmp_data/tmp/suibian/final_xgb.model")

In [1]:
from xgboost.spark import SparkXGBClassifier
from xgboost.spark import estimator


In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
# 创建二分类评估器
evaluator = BinaryClassificationEvaluator(labelCol="label")

# 计算AUC指标
auc = evaluator.evaluate(predictions)

# 打印AUC指标
print("AUC:", auc)

23/10/08 10:19:27 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.8 MiB


AUC: 0.7487723896450121


In [27]:
model.save("gs://dsp_alg_test/tmp_data/final_xgb.model")

In [20]:
model.stages[-1].get_feature_importances()

{'f0': 1314.0,
 'f1': 136.0,
 'f2': 732.0,
 'f4': 24.0,
 'f5': 511.0,
 'f6': 62.0,
 'f7': 306.0}

In [ ]:
train_predictions

In [16]:
X_test

,l1,l2,l3,l4,l5,l6,l7,l8,l9,l10,...,l21,l22,l23,l24,l25,l26,l27,l28,l29,l30
512,13.40,20.52,88.64,556.7,0.11060,0.14690,0.14450,0.08172,0.2116,0.07325,...,16.41,29.66,113.30,844.4,0.15740,0.38560,0.51060,0.20510,0.3585,0.11090
457,13.21,25.25,84.10,537.9,0.08791,0.05205,0.02772,0.02068,0.1619,0.05584,...,14.35,34.23,91.29,632.9,0.12890,0.10630,0.13900,0.06005,0.2444,0.06788
439,14.02,15.66,89.59,606.5,0.07966,0.05581,0.02087,0.02652,0.1589,0.05586,...,14.91,19.31,96.53,688.9,0.10340,0.10170,0.06260,0.08216,0.2136,0.06710
298,14.26,18.17,91.22,633.1,0.06576,0.05220,0.02475,0.01374,0.1635,0.05586,...,16.22,25.26,105.80,819.7,0.09445,0.21670,0.15650,0.07530,0.2636,0.07676
37,13.03,18.42,82.61,523.8,0.08983,0.03766,0.02562,0.02923,0.1467,0.05863,...,13.30,22.81,84.46,545.9,0.09701,0.04619,0.04833,0.05013,0.1987,0.06169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,17.42,25.56,114.50,948.0,0.10060,0.11460,0.16820,0.06597,0.1308,0.05866,...,18.07,28.07,120.40,1021.0,0.12430,0.17930,0.28030,0.10990,0.1603,0.06818
519,12.75,16.70,82.51,493.8,0.11250,0.11170,0.03880,0.02995,0.2120,0.06623,...,14.45,21.74,93.63,624.1,0.14750,0.19790,0.14230,0.08045,0.3071,0.08557
432,20.18,19.54,133.80,1250.0,0.11330,0.14890,0.21330,0.12590,0.1724,0.06053,...,22.03,25.07,146.00,1479.0,0.16650,0.29420,0.53080,0.21730,0.3032,0.08075
516,18.31,20.58,120.80,1052.0,0.10680,0.12480,0.15690,0.09451,0.1860,0.05941,...,21.86,26.20,142.20,1493.0,0.14920,0.25360,0.37590,0.15100,0.3074,0.07863


## 以下为测试

In [2]:
! pip install graphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 2.5 MB/s eta 0:00:00


In [12]:
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))
sigmoid(0-0.0857142955)

0.4785845360313352

In [34]:
for i in [0.398579329,0.124267399,-0.0319588967,-0.226236567,-0.726531267]:
    print(sigmoid(i))

0.5983462808336605
0.5310269325322643
0.49201095579498294
0.443680868684884
0.3259563780634549


In [10]:
import xgboost as xgb

# 读取模型
model = xgb.Booster()
model.load_model("./model_file_name.json")

# 测试模型
# x_test = xgb.DMatrix([[10], [20], [30]])
# predict_result = model.predict(x_test)
# print('predict_result: ', predict_result)


In [11]:
model.save_model("./demos.model")

/opt/conda/miniconda3/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [09:32:56] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [40]:
x_test = xgb.DMatrix([[13.4]])

In [23]:
x_test

In [41]:
data_dmatrix = xgb.DMatrix(data=at)


In [42]:
predict_result = model.predict(data_dmatrix)
print('predict_result: ', predict_result)


predict_result:  [0.66233516]


In [49]:
1-0.66233516

0.33766483999999997

In [ ]:
model.load_model()

In [37]:
model.dump_model('1_2_model.json')


In [52]:
clf.load_model("./xg_iris.model")

In [53]:
clf.dump_model("./xg_iris_model.josn")

In [13]:
5E-1

0.5